In [52]:
# import libraries (including our short d3_lib script)
from IPython.core.display import HTML
import d3_lib
import re
import json

In [53]:
%%script bash
arq --results JSON --data https://raw.githubusercontent.com/Planteome/plant-ontology/master/plant-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  

SELECT ?x ?xlabel ?o ?olabel
WHERE
{
    ?x rdfs:subClassOf ?o .
    filter(!isblank(?o)) 
    ?x rdfs:label ?xlabel . 
    ?o rdfs:label ?olabel .
}' > po/po.json

In [54]:
with open('po/po.json') as data_file:    
    data = json.load(data_file)
    
    list = data["results"]["bindings"]
    
    graph = {"nodes": [], "links": []}
    #force plant trait to be he first node
    #graph["nodes"].append( {"name": "plant trait", "value": 0.5, "group": 9, "id": "http://purl.obolibrary.org/obo/TO_0000387" } )
    for entry in list:
        x = 0.1
        o = 0.1
        #check if the nodes already exist in the graph
        for e in graph["nodes"]:
            if e["name"] == entry["xlabel"]["value"]:
                x=graph["nodes"].index(e)
            if e["name"] == entry["olabel"]["value"]:
                o=graph["nodes"].index(e)
        # if the term does not exist, create it        
        if x==0.1:
            graph["nodes"].append( {"name": entry["xlabel"]["value"], "value": 0.5, "group": 9, "id":entry["x"]["value"] } )
            x=len(graph["nodes"])-1
        if o==0.1:
            graph["nodes"].append( {"name": entry["olabel"]["value"], "value": 0.5, "group": 9, "id":entry["o"]["value"]} )
            o=len(graph["nodes"])-1
            
        # create the link
        if  entry["x"]["value"] != entry["o"]["value"]:
            graph["links"].append( {"source": x, "target": o, "value": 5} )



In [55]:
%%script bash
arq --results JSON --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  

SELECT ?to ?tolabel ?po
WHERE
{
    ?to (owl:equivalentClass/(owl:intersectionOf/rdf:rest*/rdf:first/owl:someValuesFrom))* ?po .
    filter (regex(str(?po), "PO_"))
    filter (?to != ?po)
    ?to rdfs:label ?tolabel
    
}' > po/topo.json

In [56]:
with open('po/topo.json') as data_file:    
    data = json.load(data_file)
    
    list = data["results"]["bindings"]
    
    for entry in list:
        x = 0.1
        o = 0.1
        #check if the nodes already exist in the graph
        for e in graph["nodes"]:
            if e["name"] == entry["tolabel"]["value"]:
                x=graph["nodes"].index(e)
            if e["id"] == entry["po"]["value"]:
                o=graph["nodes"].index(e)
                #update the size of the PO class
                e["value"]=e["value"]+0.1
        # if the term does not exist, create it        
        if x==0.1:
            graph["nodes"].append( {"name": entry["tolabel"]["value"], "value": 0.5, "group": 1, "id":entry["to"]["value"] } )
            x=len(graph["nodes"])-1
        
       
        # create the link
        if  entry["to"]["value"] != entry["po"]["value"]:
            graph["links"].append( {"source": x, "target": o, "value": 5} )


In [57]:
# visualize as force-directed graph in D3
HTML(d3_lib.set_styles(['force_directed_graph']) + 
'<script src="lib/d3/d3.min.js"></script>' + 
      d3_lib.draw_graph('po', {'data': graph}) )